In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/mobile-price-classification/train.csv")
test = pd.read_csv("/kaggle/input/mobile-price-classification/test.csv")

In [ ]:
train

In [ ]:
test

# understanding the data

In [ ]:
train.info()

In [ ]:
train.describe()

# **Saving a copy of the datasets**

In [ ]:
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
test.drop('id', axis=1, inplace=True)

# Data Processing

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

null values?

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

Feature Binning

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

In [ ]:
binner = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')

battery_capacity = 0 - low capacity, 1 - medium capacity, 2 - high capacity (or maybe its the opposite)

In [ ]:
train['battery_capacity'] = binner.fit_transform(train['battery_power'].values.reshape(-1,1)).astype('int64')
test['battery_capacity'] = binner.transform(test['battery_power'].values.reshape(-1,1)).astype('int64')

Feature Correlation

In [ ]:
corr = train.corr()
plt.figure(figsize=(25,25))
sns.heatmap(corr, cmap='coolwarm', annot=True)
plt.show()

Detecting and removing outliars

features

In [ ]:
for col in train.columns.drop('price_range'):
    plt.title(f'Boxplot of {col}')
    sns.boxplot(train[col])
    plt.show()

In [ ]:
outliar_cols = train[['fc', 'px_height']]

In [ ]:
Q1 = outliar_cols.quantile(0.25)
Q2 = outliar_cols.quantile(0.5)
Q3 = outliar_cols.quantile(0.75)

IQR = Q3-Q1
Min = Q1-(1.5*IQR)
Max = Q3+(1.5*IQR)

print("IQR : ",IQR)
print("")
print("Min : ",Min)
print("")
print("Q1 : ",Q1)
print("")
print("Q2 : ",Q2)
print("")
print("Q3 : ",Q3)
print("")
print("Max : ",Max)

In [ ]:
train = train[~((train[['fc', 'px_height']] < Min) | (train[['fc', 'px_height']] > Max)).any(axis=1)]

Scaling the data

In [ ]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler

In [ ]:
x_train, y_train = train.drop('price_range', axis=1), train['price_range']
x_test = test

In [ ]:
minmax = MinMaxScaler()
robust = RobustScaler()

for col in x_train.columns:
  x_train[col] = robust.fit_transform(x_train[col].values.reshape(-1,1))
  x_train[col] = minmax.fit_transform(x_train[col].values.reshape(-1,1))

  x_test[col] = robust.transform(x_test[col].values.reshape(-1,1))
  x_test[col] = minmax.transform(x_test[col].values.reshape(-1,1))

In [ ]:
x_train = pd.DataFrame(x_train, columns=train.columns.drop('price_range'))
x_test = pd.DataFrame(x_test, columns=test.columns)

feature imabalnce?

In [ ]:
sns.countplot(y_train)

there is no imbalance

# Splitting training data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train_, x_val, y_train_, y_val = train_test_split(x_train, y_train, test_size=0.2)

# Model Selection

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier

In [ ]:
from sklearn.metrics import mean_squared_error, accuracy_score

In [ ]:
def model_selection(x_train_, x_val, y_train_, y_val, model):
  model = model()
  model.fit(x_train_, y_train_)

  train_pred = model.predict(x_train_)
  val_pred = model.predict(x_val)

  val_error = np.sqrt(mean_squared_error(y_val, val_pred))
  train_error = np.sqrt(mean_squared_error(y_train_, train_pred))
  train_acc = accuracy_score(y_train_, train_pred)
  val_acc = accuracy_score(y_val, val_pred)

  print('Train Error:', train_error)
  print('\n')
  print('Val Error:', val_error)
  print('\n')
  print('Train ACC :', train_acc*100)
  print('\n')
  print('Val ACC :', val_acc*100)
  print('\n')
  print('Is overfitting:', True if train_acc>val_acc else False)
  print('\n')
  print('Overfitting by:',train_acc*100-val_acc*100)

In [ ]:
extratrees = model_selection(x_train_, x_val, y_train_, y_val, ExtraTreesClassifier)
extratrees

In [ ]:
gradient = model_selection(x_train_, x_val, y_train_, y_val, GradientBoostingClassifier)
gradient

In [ ]:
randomforest = model_selection(x_train_, x_val, y_train_, y_val, RandomForestClassifier)
randomforest

In [ ]:
ada = model_selection(x_train_, x_val, y_train_, y_val, AdaBoostClassifier)
ada

In [ ]:
xgb = model_selection(x_train_, x_val, y_train_, y_val, XGBClassifier)
xgb

In [ ]:
lgbm = model_selection(x_train_, x_val, y_train_, y_val, LGBMClassifier)
lgbm

In [ ]:
tree = model_selection(x_train_, x_val, y_train_, y_val, DecisionTreeClassifier)
tree

In [ ]:
catboost = model_selection(x_train_, x_val, y_train_, y_val, CatBoostClassifier)
catboost

In [ ]:
sgd = model_selection(x_train_, x_val, y_train_, y_val, SGDClassifier)
sgd

i will choose CatBoostClassifier

# Model building and training

In [ ]:
model = CatBoostClassifier()
model.fit(x_train, y_train)

# Predictions

In [ ]:
pred = model.predict(x_test)
pred

In [ ]:
Pred = pd.DataFrame(pred, columns=['price_range'])
Pred

In [ ]:
sns.countplot(Pred['price_range'])

In [ ]:
IDS = test_copy['id']

In [ ]:
Predictions = pd.concat([IDS, Pred], axis=1)
Predictions

# Saving the predictions file

In [ ]:
Predictions.to_csv("/kaggle/working/Predictions.csv", index=False)